In [ ]:
# Mocking data for dev

In [ ]:
import json
import pandas as pd
from faker import Faker

In [ ]:
post_code_df = pd.read_csv('_62_postcode.csv').drop_duplicates(subset=['จังหวัด','อำเภอ'])
electoral_district_df = pd.read_csv('_66_Vote62_election-zones.csv')

In [ ]:
electoral_district_df

In [ ]:
district_list = list()
electdistrict = dict()
for g, g_df in electoral_district_df.groupby(['จังหวัด', 'อำเภอ']):
    province, district = g
    electoral_nums = g_df['เขตเลือกตั้ง.1'].drop_duplicates().astype(int)
    districts = g_df['อำเภอ'].drop_duplicates().values

    dks = [f'{province}-{el_n}' for el_n in electoral_nums]
    
    for dk, en in zip(dks, electoral_nums):
        d = electdistrict.get(dk, dict(district_list=list(), province=province, electoralDistrictNumber=en))
        d['district_list'] += districts.tolist()
        electdistrict[dk] = d
    for d in districts:
        district_list.append(
            dict(district=d, province=province, electoral=[dict(fk=fk) for fk in dks]))

In [ ]:
with open('district_province_list.json', 'w') as fp:
    json.dump(district_list, fp, ensure_ascii=False)

In [ ]:
with open('electoral_district_table.json', 'w') as fp:
    json.dump(electdistrict, fp, ensure_ascii=False)

In [ ]:
import random
import re
import requests
import os
fake = Faker('th')

In [ ]:
electoral_dir = 'electorals'
os.makedirs(electoral_dir, exist_ok=True)

In [ ]:
twfu_people=pd.read_csv('./_People_table_exported_1.csv')
twfu_parties=pd.read_csv('./_Parties_table_exported_1.csv')

In [ ]:
twfu_parties = twfu_parties[(twfu_parties.PartyType == 'พรรค') & (~twfu_parties.Images.isna())]
twfu_people  = twfu_people[~twfu_people.IsSenator]

In [ ]:
image_urls = twfu_people[~twfu_people.Images.isna()].Images.apply(lambda x: re.sub('.*\((.*)\)','\\1',x)).tolist()
occupations = twfu_people[~twfu_people.ExOccupation0.isna()].ExOccupation0.unique().astype(str).tolist()

In [ ]:
party_list = []
total_party = (twfu_parties.PartyType == 'พรรค').sum()
party_number = list(range(1, total_party+1))
random.shuffle(party_number)
def get_url(url_pattern, party_name):
    url =  url_pattern.format(party_name)
    if requests.get(url).ok:
        return url
    return ''

for i, party in twfu_parties[twfu_parties.PartyType == 'พรรค'].iterrows():
    candiate_temp = []
    for i in range(random.randint(1,3)):
        candiate_temp.append(dict(Name=fake.name(), Image=random.choice(image_urls)))
    
    party_list.append(
        dict(Number=party_number.pop(),
             Image=re.sub('.*\((.*)\)','\\1', party.Images) if isinstance(party.Images, str) else '',
             Name=party.Name,
             PartyGroup=random.choice(['ฝ่ายค้าน', 'ฝ่ายรัฐบาล', 'ทั้งฝ่ายรัฐบาลและฝ่ายค้าน']),
             Candidate=candiate_temp,
             PastGovernment=random.random() < 0.1,
             PastOpposition=random.random() < 0.8,
             Policy=get_url('https://staging.election66.wevis.info/policyshop/party/{}',party.Name),
             Promise=get_url('https://promisetracker.wevis.info/explore?party={}',party.Name),
             Law='https://wevis.info/law-watch#investigate-section',
             Others=get_url('https://theyworkforus.wevis.info/party/{}', party.Name),
             Website=party.Website if isinstance(party.Website, str) else '',
             PartyList=[dict(Name=fake.name(), PartyListNumber=plnum) for plnum in range(1,101)],
            ))

In [ ]:
party_dict = {p['Name']: p for p in party_list}

In [ ]:
people_list = []
for (province, electoral_nums), g_df in electoral_district_df.groupby(['จังหวัด', 'เขตเลือกตั้งที่']):
    is_in_province = twfu_people['MpProvince'] == province
    total_people=random.randint(10, 30)
    parties = twfu_parties.sample(total_people)['Name'].to_list()
    electoral_people_list = []
    electoral_parties = []
    for i in range(1, total_people+1):
        person = dict(
            Name=fake.name() if random.random() < 0.5 else random.choice(twfu_people.Name.tolist()),
            Birthdate=fake.iso8601().split('T')[0],
            Number=i,
            Education=random.choice(['ปริญญาตรี', 'ปริญญาโท', 'ปริญญาเอก', 'ปริญญาจัตวา', 'ด้อกเต้อ', 'อิฟลู']),
            ExOccupation=random.choice(occupations + ['สมาชิกสภาผู้แทนราษฎร']),
            Party=parties.pop(),
            Province=province,
            Zone=int(electoral_nums),
            Image=(random.choice(image_urls) if random.random() < 0.5 else ''),
            PastGovernment=random.random() < 0.1,
        )
        person['PastOpposition'] = (not person['PastGovernment']) if random.random() < 0.8 else person['PastGovernment']
        electoral_parties.append(person['Party'])
        people_list.append(person)
        electoral_people_list.append(person)

    file_path = os.path.join(electoral_dir,f'{province}-{electoral_nums}.json')
    electoral_data = dict(People=electoral_people_list,Parties=electoral_parties)

    with open(file_path,'w') as fp:
        json.dump(electoral_data, fp, ensure_ascii=False)

In [ ]:
# with open('people_list.json', 'w') as fp:
#     json.dump(people_list, fp, ensure_ascii=False)
with open('parties.json', 'w') as fp:
    json.dump(party_dict, fp, ensure_ascii=False)